## Introdução

Neste projeto, desenvolvemos um sistema para previsão de resultados de partidas de League of Legends (LoL) no Campeonato Brasileiro (CBLoL) utilizando técnicas de web scraping e aprendizado de máquina. Nosso objetivo é construir um modelo preditivo que, com base em dados históricos e estatísticas dos jogos, seja capaz de prever o resultado de futuras partidas. Além disso, extraímos métricas que ajudam a entender o desempenho de cada time e jogador, fornecendo insights valiosos para análise estratégica.

Importando as bibliotecas necessárias

In [109]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict
import time
import os
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import numpy as np
import requests

RANDOM_SEED = 42

Extração dos urls das páginas com os vídeos e estatísticas das partidas

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://lolesports.com/pt-BR/vods/cblol-brazil/cblol_split_1_2024'
driver.get(url)

time.sleep(5)

video_url = []

base_xpath = '//*[@id="vodsRoot"]/div[2]/div[1]/div[{}]/div/div[2]/div[1]/div[{}]/*'

i, j = 2, 1 
tentativas = 0  

while True:
    try:
        button_xpath = base_xpath.format(i, j)
        button = driver.find_element(By.XPATH, button_xpath)
        video_url.append(button.get_attribute('href'))
        j += 1
        tentativas = 0
    except:
        if tentativas < 2:
            i += 1
            j = 1  
            tentativas += 1
        else:
            break

# Fecha o navegador
driver.quit()

In [110]:
# URL do site
url = "https://lolesports.com/pt-BR/vods/cblol-brazil/cblol_split_1_2024"

# Enviando requisição HTTP para obter o conteúdo da página
response = requests.get(url)
content = response.content

# Analisando o conteúdo da página com BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Lista para armazenar os links
links = []

# Encontrando todos os elementos 'a' com as classes específicas
a_tags = soup.find_all('a', class_="d_inline-flex w_38 h_48 justify_center items_center text_text.secondary border-b_4px_solid_{colors.text.secondary} z_zLayer2 hover:text_accent.primary hover:border-b_accent.primary spoilerHidden:vis_hidden")

# Iterando sobre os elementos encontrados e extraindo os links
for tag in a_tags:
    href = tag.get('href')
    if href:
        full_link = f"https://lolesports.com{href}"
        links.append(full_link)

# Revertendo a lista para que os links sejam listados de baixo para cima
links.reverse()

print(f"Total de links encontrados: {len(links)}")

Total de links encontrados: 130


Função para extrair o html de cada jogo, percorrendo a lista de urls e salvando o tempo da partida de acordo com a parte do vídeo (early, mid e late game):

In [111]:
# Get the current working directory
current_directory = os.getcwd()

driver = webdriver.Chrome()
count = 1

def get_game_data(driver, count, game_time):
        #switch case to game_time ("early", "mid", "late")
        if game_time == "early":
            actions.send_keys(Keys.NUMPAD4).perform()
            time.sleep(8)
        elif game_time == "mid":
            actions.send_keys(Keys.NUMPAD6).perform()
            time.sleep(8)
        elif game_time == "late":
            actions.send_keys(Keys.NUMPAD9).perform()
            time.sleep(8)

        #get the htlm of the page
        html = driver.page_source

        file_name = f"{game_time}_lolesports_{count}.txt"
        # Obter o diretório do script atual
        script_directory = os.path.dirname(os.path.abspath(current_directory))
        
        # Construir o caminho completo para o arquivo 'lolesports.txt' dentro da pasta 'data/raw'
        file_path = os.path.join(script_directory, 'data', 'raw', file_name)
        # Garantir que o diretório 'data/raw' exista
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        # Salvar a saída no arquivo
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(html)

for url in links:
    try:
        driver.get(url)
        
        actions = ActionChains(driver)

        # print("Waiting for video to load")
        time.sleep(5)
        # print("Video loaded")

        start_button = driver.find_element(By.XPATH, "//*[@class='attached']")
        start_button.click()

        time.sleep(5)

        get_game_data(driver, count, "early")
        get_game_data(driver, count, "mid")
        get_game_data(driver, count, "late")
        
        print(f"Game {count} done")
        count += 1
    except Exception as e:
        print(f"Erro no count {count}: {e}")
        count += 1
        
print("Closing driver")
driver.quit()



Game 1 done
Game 2 done
Game 3 done
Game 4 done
Game 5 done
Game 6 done
Game 7 done
Game 8 done
Game 9 done
Game 10 done
Game 11 done
Game 12 done
Game 13 done
Game 14 done
Game 15 done
Game 16 done
Game 17 done
Game 18 done
Game 19 done
Game 20 done
Game 21 done
Game 22 done
Game 23 done
Game 24 done
Game 25 done
Game 26 done
Game 27 done
Game 28 done
Game 29 done
Game 30 done
Game 31 done
Game 32 done
Game 33 done
Game 34 done
Game 35 done
Game 36 done
Game 37 done
Game 38 done
Game 39 done
Game 40 done
Game 41 done
Game 42 done
Game 43 done
Game 44 done
Game 45 done
Game 46 done
Game 47 done
Game 48 done
Game 49 done
Game 50 done
Game 51 done
Game 52 done
Game 53 done
Game 54 done
Game 55 done
Game 56 done
Game 57 done
Game 58 done
Game 59 done
Game 60 done
Game 61 done
Game 62 done
Game 63 done
Game 64 done
Game 65 done
Game 66 done
Game 67 done
Game 68 done
Game 69 done
Game 70 done
Game 71 done
Game 72 done
Game 73 done
Game 74 done
Game 75 done
Game 76 done
Game 77 done
Game 78 

In [112]:
# Obter o diretório do script atual
script_directory = os.path.dirname(os.path.abspath(current_directory))

def get_html_file(file_name):
    # Construir o caminho completo para o arquivo 'lolesports.txt' dentro da pasta 'data/raw'
    file_path = os.path.join(script_directory, 'data', 'raw', file_name)

    # Abrir o arquivo
    with open(file_path, 'r', encoding='utf-8') as file:
        html = file.read()
    
    return html

def get_lane_players_stats(soup, lane):

    if lane == 'top':
        # Tratamento especial devido as classes obtidas como o crawler
        lane_players = soup.find_all('div', {'class': f'player top selected primary'})
        lane_players += soup.find_all('div', {'class': f'player top selected primary dead'})
        lane_players += soup.find_all('div', {'class': f'player top selected secondary'})
        lane_players += soup.find_all('div', {'class': f'player top selected secondary dead'})

    else:
        # Encontrar todos os elementos com a classe 'player {lane}' para ambos os times
        lane_players = soup.find_all('div', {'class': f'player {lane}'})

        # Encontrar todos os elementos com a classe 'player {lane} dead' para ambos os time
        lane_players += soup.find_all('div', {'class': f'player {lane} dead'})

    # Iterar sobre os jogadores encontrados
    player_stats = []
    for lane_player in lane_players:
        # Obter o nome do jogador
        player_name = lane_player.find('div', {'class': 'name'}).text
        player_name = player_name.split(" ")[1]

        # Obter as estatísticas de kills, deaths e assists
        kda_element = lane_player.find('div', {'class': 'stat kda'})
        kills = kda_element.find('span', {'class': 'kills'}).text
        deaths = kda_element.find('span', {'class': 'deaths'}).text
        assists = kda_element.find('span', {'class': 'assists'}).text

        # Obter as estatísticas de cs e gold
        cs_element = lane_player.find('div', {'class': 'stat cs'})
        cs = cs_element.text

        gold_element = lane_player.find('div', {'class': 'stat gold'})
        gold = gold_element.text

        player_stats = [player_name, kills, deaths, assists, cs, gold]
        # print(f"{player_name} KDA: {kills}/{deaths}/{assists} CS: {cs} Gold: {gold}")

    return player_stats

def get_team_name(soup):
    lane_players = soup.find_all('div', {'class': f'player mid'})
    lane_players += soup.find_all('div', {'class': f'player mid dead'})

    for lane_player in lane_players:
        # Obter o nome do jogador
        player_name = lane_player.find('div', {'class': 'name'}).text

        team_name = player_name.split(" ")[0]

    return team_name

lanes = ['top', 'jungle', 'mid', 'bottom', 'support']
# lanes = ['bottom']

# Construir o caminho completo para o arquivo 'data.csv' dentro da pasta 'data/processed'
file_path = os.path.join(script_directory, 'data', 'processed', 'data.csv')

# Garantir que o diretório 'data/processed' exista
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Create a cvs file to save the data
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["game","game_time",
                     "blue_team", "blue_team_dragons", "blue_team_gold", "blue_team_inhibitors", "blue_team_barons", "blue_team_towers", "blue_team_kills",
                     "blue_team_top", "blue_team_top_kills", "blue_team_top_deaths", "blue_team_top_assists", "blue_team_top_cs", "blue_team_top_gold",
                     "blue_team_jungle", "blue_team_jungle_kills", "blue_team_jungle_deaths", "blue_team_jungle_assists", "blue_team_jungle_cs", "blue_team_jungle_gold",
                     "blue_team_mid", "blue_team_mid_kills", "blue_team_mid_deaths", "blue_team_mid_assists", "blue_team_mid_cs", "blue_team_mid_gold",
                     "blue_team_bottom", "blue_team_bottom_kills", "blue_team_bottom_deaths", "blue_team_bottom_assists", "blue_team_bottom_cs", "blue_team_bottom_gold",
                     "blue_team_support", "blue_team_support_kills", "blue_team_support_deaths", "blue_team_support_assists", "blue_team_support_cs", "blue_team_support_gold",
                     "red_team", "red_team_dragons", "red_team_gold", "red_team_inhibitors", "red_team_barons", "red_team_towers", "red_team_kills",
                     "red_team_top", "red_team_top_kills", "red_team_top_deaths", "red_team_top_assists", "red_team_top_cs", "red_team_top_gold",
                     "red_team_jungle", "red_team_jungle_kills", "red_team_jungle_deaths", "red_team_jungle_assists", "red_team_jungle_cs", "red_team_jungle_gold",
                     "red_team_mid", "red_team_mid_kills", "red_team_mid_deaths", "red_team_mid_assists", "red_team_mid_cs", "red_team_mid_gold",
                     "red_team_bottom", "red_team_bottom_kills", "red_team_bottom_deaths", "red_team_bottom_assists", "red_team_bottom_cs", "red_team_bottom_gold",
                     "red_team_support", "red_team_support_kills", "red_team_support_deaths", "red_team_support_assists", "red_team_support_cs", "red_team_support_gold"
                     ])

#ler cada arquivo da pasta raw
# files = ["late_lolesports_1.txt"]
for file_name in os.listdir(os.path.join(script_directory, 'data', 'raw')):
# for file_name in files:
    # Ignorar arquivos que não terminam com '.txt'
    if not file_name.endswith('.txt'):
        continue

    # Obter o html do arquivo
    html = get_html_file(file_name)

    # Criar o objeto BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # print(f"Arquivo: {file_name}")
    # print(" ")
    game_time = file_name.split("_")[0]

    stats_team_summary = soup.find('div', {'class': 'StatsTeamsSummary'})
    if stats_team_summary is None:
        print(f"Arquivo: {file_name} não possui a tag StatsTeamsSummary")
        continue
    
    # Get dragons stats
    dragons = stats_team_summary.find('div', {'class': 'dragons'})

    blue_team_dragons = dragons.find_all('div', {'class': 'blue-team'})
    red_team_dragons = dragons.find_all('div', {'class': 'red-team'})

    blue_team_dragons_list = []
    for blue_team_dragon in blue_team_dragons:

        dragon_divs = blue_team_dragon.find_all('div')

        for div in dragon_divs:
            if 'class' in div.attrs:
                classes = div['class']
                blue_team_dragons_list.append(classes[1])
    
    red_team_dragons_list = []
    for red_team_dragon in red_team_dragons:

        dragon_divs = red_team_dragon.find_all('div')

        for div in dragon_divs:
            if 'class' in div.attrs:
                classes = div['class']
                red_team_dragons_list.append(classes[1])
    
    # Get gold stats
    gold = stats_team_summary.find('div', {'class': 'gold'})
    totals = gold.find('div', {'class': 'totals'})
    blue_team_gold = totals.find('div', {'class': 'blue-team'}).text
    red_team_gold = totals.find('div', {'class': 'red-team'}).text

    # Get details
    details = stats_team_summary.find('div', {'class': 'details'})
    blue_team_details = details.find('div', {'class': 'blue-team'})
    blue_team_inhibitors = blue_team_details.find('div', {'class': 'stat inhibitors'}).text
    blue_team_barons = blue_team_details.find('div', {'class': 'stat barons'}).text
    blue_team_towers = blue_team_details.find('div', {'class': 'stat towers'}).text
    blue_team_kills = blue_team_details.find('div', {'class': 'stat kills'}).text

    red_team_details = details.find('div', {'class': 'red-team'})
    red_team_inhibitors = red_team_details.find('div', {'class': 'stat inhibitors'}).text
    red_team_barons = red_team_details.find('div', {'class': 'stat barons'}).text
    red_team_towers = red_team_details.find('div', {'class': 'stat towers'}).text
    red_team_kills = red_team_details.find('div', {'class': 'stat kills'}).text

    stats_team_playes = soup.find('div', {'class': 'StatsTeamsPlayers'})

    blue_team = stats_team_playes.find('div', {'class': 'blue-team'})
    blue_team_name = get_team_name(blue_team)
    blue_team_players_stats = []
    for lane in lanes:
        blue_team_players_stats += get_lane_players_stats(blue_team, lane)

    red_team = stats_team_playes.find('div', {'class': 'red-team'})
    red_team_name = get_team_name(red_team)

    red_team_players_stats = []
    for lane in lanes:
        red_team_players_stats += get_lane_players_stats(red_team, lane)
    game_id = file_name.split("_")[2].split(".")[0]
    
    # Write the data in the csv file
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([game_id, game_time,
                         blue_team_name, blue_team_dragons_list, blue_team_gold, blue_team_inhibitors, blue_team_barons, blue_team_towers, blue_team_kills] + blue_team_players_stats +
                         [red_team_name, red_team_dragons_list, red_team_gold, red_team_inhibitors, red_team_barons, red_team_towers, red_team_kills] + red_team_players_stats
                        )

Arquivo: early_lolesports_122.txt não possui a tag StatsTeamsSummary
Arquivo: early_lolesports_123.txt não possui a tag StatsTeamsSummary
Arquivo: early_lolesports_124.txt não possui a tag StatsTeamsSummary
Arquivo: early_lolesports_125.txt não possui a tag StatsTeamsSummary
Arquivo: early_lolesports_61.txt não possui a tag StatsTeamsSummary
Arquivo: late_lolesports_122.txt não possui a tag StatsTeamsSummary
Arquivo: late_lolesports_123.txt não possui a tag StatsTeamsSummary
Arquivo: late_lolesports_124.txt não possui a tag StatsTeamsSummary
Arquivo: late_lolesports_125.txt não possui a tag StatsTeamsSummary
Arquivo: mid_lolesports_122.txt não possui a tag StatsTeamsSummary
Arquivo: mid_lolesports_123.txt não possui a tag StatsTeamsSummary
Arquivo: mid_lolesports_124.txt não possui a tag StatsTeamsSummary
Arquivo: mid_lolesports_125.txt não possui a tag StatsTeamsSummary


In [148]:
# Obtendo o diretório atual do notebook
notebook_dir = %pwd

# Construindo o caminho completo para data.csv
file_path = os.path.join(notebook_dir, '..', 'data', 'processed', 'data.csv')

In [149]:
# Lendo o arquivo CSV usando pandas
data_raw = pd.read_csv(file_path)

data = data_raw.copy()

#Exibe as 5 primeiras linhas do dataframe
data.head(12)

,game,game_time,blue_team,blue_team_dragons,blue_team_gold,blue_team_inhibitors,blue_team_barons,blue_team_towers,blue_team_kills,blue_team_top,...,red_team_bottom_deaths,red_team_bottom_assists,red_team_bottom_cs,red_team_bottom_gold,red_team_support,red_team_support_kills,red_team_support_deaths,red_team_support_assists,red_team_support_cs,red_team_support_gold
0,1,early,RED,[],0,0,0,0,0,fNb,...,0,0,0,0,Kuri,0,0,0,0,0
1,10,early,LOS,['infernal'],20.3 K,0,0,1,5,SuperCleber,...,2,0,55,3.1 K,scamber,0,2,0,13,2.4 K
2,100,early,PNG,[],24.5 K,0,0,1,7,Wizer,...,1,1,129,4.9 K,ProDelta,1,3,2,21,3.6 K
3,101,early,VKS,['infernal'],19.7 K,0,0,0,2,Guigo,...,1,0,105,4.4 K,Kuri,2,0,1,20,3.8 K
4,102,early,PNG,[],2.5 K,0,0,0,0,Wizer,...,0,0,0,500,ProDelta,0,0,0,0,500
5,103,early,LLL,['mountain'],35.8 K,0,0,2,9,Robo,...,1,1,165,8.6 K,JoJo,0,2,6,20,4.8 K
6,104,early,RED,[],2.5 K,0,0,0,0,fNb,...,0,0,0,500,RedBert,0,0,0,0,500
7,105,early,LLL,[],27.7 K,0,0,1,7,Robo,...,2,1,131,5.4 K,JoJo,1,3,1,25,3.8 K
8,106,early,KBM,[],27.0 K,0,0,1,2,Lonely,...,0,4,169,7.6 K,JoJo,0,0,6,22,4.0 K
9,107,early,KBM,[],0,0,0,0,0,Lonely,...,0,0,0,0,JoJo,0,0,0,0,0


In [150]:
#Drop the game 122, 123, 124, 125 and 61
data = data.drop(data[(data['game'] == 122) | (data['game'] == 123) | (data['game'] == 124) | (data['game'] == 125) | (data['game'] == 61)].index)

In [151]:
# ordena o dataframe pelo id do jogo, de forma decrescente
data = data.sort_values(by='game')

#printa o game, blue_team e red_team
data[['game', 'blue_team', 'red_team']].head(12)

,game,blue_team,red_team
0,1,RED,PNG
251,1,RED,PNG
125,1,RED,PNG
38,2,LBR,ITZ
289,2,LBR,ITZ
163,2,LBR,ITZ
49,3,FX,FUR
300,3,FX,FUR
174,3,FX,FUR
60,4,VKS,LOS


In [152]:
# URL da página com a tabela
url = "https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1/Match_History"

# Realiza a requisição HTTP para obter o conteúdo da página
response = requests.get(url)
html_content = response.content

# Usa o BeautifulSoup para fazer o parsing do HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Encontra todas as linhas de jogos na tabela
rows = soup.find_all('tr', class_=['mhgame-blue', 'mhgame-red'])

# Dicionário para armazenar os vencedores agrupados por data
games_by_date = defaultdict(list)

# Itera sobre cada linha para extrair os dados necessários
for row in rows:
    # Encontra todas as colunas na linha
    columns = row.find_all('td', class_='mhgame-result')
    
    if columns:
        # O primeiro elemento na lista de colunas contém a data
        date_column = columns[0]
        game_date = date_column.text.strip()

        # O terceiro elemento (index 4) na lista de colunas contém o vencedor
        winner_column = columns[4]
        winner_team = winner_column.find('a').get('data-to-id')
        
        # Adiciona o jogo à lista correspondente à data
        games_by_date[game_date].append(winner_team)

# Dicionário final de vencedores com a ordem ajustada
winners = {}
index = 1

# Itera sobre os jogos agrupados por data
for game_date, games in games_by_date.items():
    # Inverte a ordem dos jogos dentro do mesmo dia
    games.reverse()
    # Adiciona os jogos ao dicionário final
    for game in games:
        winners[index] = game
        index += 1

In [153]:
# URL da página com a tabela
url = "https://lol.fandom.com/wiki/CBLOL/2024_Season/Split_1_Playoffs/Match_History"

# Realiza a requisição HTTP para obter o conteúdo da página
response = requests.get(url)
html_content = response.content

# Usa o BeautifulSoup para fazer o parsing do HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Encontra todas as linhas de jogos na tabela
rows = soup.find_all('tr', class_=['mhgame-blue', 'mhgame-red'])

# Dicionário para armazenar os vencedores agrupados por data
games_by_date = defaultdict(list)

# Itera sobre cada linha para extrair os dados necessários
for row in rows:
    # Encontra todas as colunas na linha
    columns = row.find_all('td', class_='mhgame-result')
    
    if columns:
        # O primeiro elemento na lista de colunas contém a data
        date_column = columns[0]
        game_date = date_column.text.strip()

        # O terceiro elemento (index 4) na lista de colunas contém o vencedor
        winner_column = columns[4]
        winner_team = winner_column.find('a').get('data-to-id')
        
        # Adiciona o jogo à lista correspondente à data
        games_by_date[game_date].append(winner_team)

winners_playoffs = {}
# Itera sobre os jogos agrupados por data
for game_date, games in games_by_date.items():
    
    # Adiciona os jogos ao dicionário final
    for game in games:
        winners_playoffs[index] = game
        index += 1

In [154]:
# Lista das equipes vencedoras
winning_teams = list(winners.values())
winning_teams_playoffs = list(winners_playoffs.values())

# Inverte a ordem das equipes vencedoras
inverted_winning_teams = winning_teams[::-1]
inverted_winning_teams_playoffs = winning_teams_playoffs[::-1]

# Cria um novo dicionário com as equipes vencedoras invertidas
winners = {game_number: inverted_winning_teams[i] for i, game_number in enumerate(range(1, 91))}
inverted_winners_playoffs = {game_number: inverted_winning_teams_playoffs[i] for i, game_number in enumerate(range(91, 131))}

# unir os dicionários de vencedores
winners.update(inverted_winners_playoffs)

In [159]:
#troca o nome do time pela sua sigla
for i, winner in winners.items():
    if winner == "RED_Canids":
        winners[i] = "RED"
    elif winner == "Liberty":
        winners[i] = "LBR"
    elif winner == "Vivo_Keyd_Stars":
        winners[i] = "VKS"
    elif winner == "PaiN_Gaming":
        winners[i] = "PNG"
    elif winner == "Los_Grandes":
        winners[i] = "LOS"
    elif winner == "INTZ":
        winners[i] = "ITZ"
    elif winner == "KaBuM_21-_Esports":
        winners[i] = "KBM"
    elif winner == "LOUD":
        winners[i] = "LLL"
    elif winner == "FURIA":
        winners[i] = "FUR"
    elif winner == "Fluxo":
        winners[i] = "FX"

In [160]:
# Adiciona coluna team_winner usando a coluna "game" como chave para o dicionário winners
data['team_winner'] = data['game'].map(winners)

In [162]:
# print game, blue_team, red_team and team_winner
# data[['game', 'blue_team', 'red_team', 'team_winner']].head(12)
data.head(12)

,game,game_time,blue_team,blue_team_dragons,blue_team_gold,blue_team_inhibitors,blue_team_barons,blue_team_towers,blue_team_kills,blue_team_top,...,red_team_bottom_assists,red_team_bottom_cs,red_team_bottom_gold,red_team_support,red_team_support_kills,red_team_support_deaths,red_team_support_assists,red_team_support_cs,red_team_support_gold,team_winner
0,1,early,RED,[],0,0,0,0,0,fNb,...,0,0,0,Kuri,0,0,0,0,0,RED
251,1,mid,RED,[],0,0,0,0,0,fNb,...,0,0,0,Kuri,0,0,0,0,0,RED
125,1,late,RED,[],0,0,0,0,0,fNb,...,0,0,0,Kuri,0,0,0,0,0,RED
38,2,early,LBR,[],33.8 K,0,0,2,4,Makes,...,4,200,7.7 K,Damage,0,1,6,26,4.9 K,LBR
289,2,mid,LBR,[],45.3 K,0,1,3,6,Makes,...,4,241,10.0 K,Damage,0,2,8,28,6.1 K,LBR
163,2,late,LBR,"['elder', 'elder']",82.1 K,2,2,9,18,Makes,...,9,377,17.9 K,Damage,1,6,15,31,10.2 K,LBR
49,3,early,FX,['ocean'],12.9 K,0,0,0,2,Kiari,...,0,61,2.5 K,zay,0,0,0,10,1.9 K,FX
300,3,mid,FX,"['ocean', 'cloud']",28.9 K,0,0,0,7,Kiari,...,0,153,5.5 K,zay,0,0,0,22,3.7 K,FX
174,3,late,FX,"['ocean', 'cloud', 'hextech', 'hextech']",56.1 K,2,1,10,23,Kiari,...,0,232,8.6 K,zay,0,3,1,32,5.6 K,FX
60,4,early,VKS,['chemtech'],19.5 K,0,0,0,2,Guigo,...,0,103,3.9 K,Kabbie,0,1,0,21,2.7 K,VKS


In [163]:
# Transform column blue_team_dragons and red_team_dragons in numeric columns counting the number of each dragon
data['blue_hextec_dragon'] = data['blue_team_dragons'].str.count('hextech')
data['blue_mountain_dragon'] = data['blue_team_dragons'].str.count('mountain')
data['blue_cloud_dragon'] = data['blue_team_dragons'].str.count('cloud')
data['blue_ocean_dragon'] = data['blue_team_dragons'].str.count('ocean')
data['blue_infernal_dragon'] = data['blue_team_dragons'].str.count('infernal')
data['blue_chemtech_dragon'] = data['red_team_dragons'].str.count('chemtech')
#drop column blue_team_dragons
data.drop('blue_team_dragons', axis=1, inplace=True)

data['red_hextec_dragon'] = data['red_team_dragons'].str.count('hextech')
data['red_mountain_dragon'] = data['red_team_dragons'].str.count('mountain')
data['red_cloud_dragon'] = data['red_team_dragons'].str.count('cloud')
data['red_ocean_dragon'] = data['red_team_dragons'].str.count('ocean')
data['red_infernal_dragon'] = data['red_team_dragons'].str.count('infernal')
data['red_chemtech_dragon'] = data['red_team_dragons'].str.count('chemtech')
#drop column red_team_dragons
data.drop('red_team_dragons', axis=1, inplace=True)

data.head(12)

,game,game_time,blue_team,blue_team_gold,blue_team_inhibitors,blue_team_barons,blue_team_towers,blue_team_kills,blue_team_top,blue_team_top_kills,...,blue_cloud_dragon,blue_ocean_dragon,blue_infernal_dragon,blue_chemtech_dragon,red_hextec_dragon,red_mountain_dragon,red_cloud_dragon,red_ocean_dragon,red_infernal_dragon,red_chemtech_dragon
0,1,early,RED,0,0,0,0,0,fNb,0,...,0,0,0,0,0,0,0,0,0,0
251,1,mid,RED,0,0,0,0,0,fNb,0,...,0,0,0,0,0,0,0,0,0,0
125,1,late,RED,0,0,0,0,0,fNb,0,...,0,0,0,0,0,0,0,0,0,0
38,2,early,LBR,33.8 K,0,0,2,4,Makes,0,...,0,0,0,1,1,0,0,0,1,1
289,2,mid,LBR,45.3 K,0,1,3,6,Makes,1,...,0,0,0,1,2,0,0,0,1,1
163,2,late,LBR,82.1 K,2,2,9,18,Makes,3,...,0,0,0,1,2,0,0,0,1,1
49,3,early,FX,12.9 K,0,0,0,2,Kiari,0,...,0,1,0,0,0,0,0,0,0,0
300,3,mid,FX,28.9 K,0,0,0,7,Kiari,1,...,1,1,0,0,0,0,0,0,0,0
174,3,late,FX,56.1 K,2,1,10,23,Kiari,4,...,1,1,0,0,0,0,0,0,0,0
60,4,early,VKS,19.5 K,0,0,0,2,Guigo,0,...,0,0,0,0,0,1,0,0,0,0


In [164]:
# Remove " K" from the end of the string and converts to numeric
data['blue_team_gold'] = data['blue_team_gold'].str.replace('K', '').astype(float)
data['blue_team_top_gold'] = data['blue_team_top_gold'].str.replace('K', '').astype(float)
data['blue_team_jungle_gold'] = data['blue_team_jungle_gold'].str.replace('K', '').astype(float)
data['blue_team_mid_gold'] = data['blue_team_mid_gold'].str.replace('K', '').astype(float)
data['blue_team_bottom_gold'] = data['blue_team_bottom_gold'].str.replace('K', '').astype(float)
data['blue_team_support_gold'] = data['blue_team_support_gold'].str.replace('K', '').astype(float)

data['red_team_gold'] = data['red_team_gold'].str.replace('K', '').astype(float)
data['red_team_top_gold'] = data['red_team_top_gold'].str.replace('K', '').astype(float)
data['red_team_jungle_gold'] = data['red_team_jungle_gold'].str.replace('K', '').astype(float)
data['red_team_mid_gold'] = data['red_team_mid_gold'].str.replace('K', '').astype(float)
data['red_team_bottom_gold'] = data['red_team_bottom_gold'].str.replace('K', '').astype(float)
data['red_team_support_gold'] = data['red_team_support_gold'].str.replace('K', '').astype(float)

data.head(12)

,game,game_time,blue_team,blue_team_gold,blue_team_inhibitors,blue_team_barons,blue_team_towers,blue_team_kills,blue_team_top,blue_team_top_kills,...,blue_cloud_dragon,blue_ocean_dragon,blue_infernal_dragon,blue_chemtech_dragon,red_hextec_dragon,red_mountain_dragon,red_cloud_dragon,red_ocean_dragon,red_infernal_dragon,red_chemtech_dragon
0,1,early,RED,0.0,0,0,0,0,fNb,0,...,0,0,0,0,0,0,0,0,0,0
251,1,mid,RED,0.0,0,0,0,0,fNb,0,...,0,0,0,0,0,0,0,0,0,0
125,1,late,RED,0.0,0,0,0,0,fNb,0,...,0,0,0,0,0,0,0,0,0,0
38,2,early,LBR,33.8,0,0,2,4,Makes,0,...,0,0,0,1,1,0,0,0,1,1
289,2,mid,LBR,45.3,0,1,3,6,Makes,1,...,0,0,0,1,2,0,0,0,1,1
163,2,late,LBR,82.1,2,2,9,18,Makes,3,...,0,0,0,1,2,0,0,0,1,1
49,3,early,FX,12.9,0,0,0,2,Kiari,0,...,0,1,0,0,0,0,0,0,0,0
300,3,mid,FX,28.9,0,0,0,7,Kiari,1,...,1,1,0,0,0,0,0,0,0,0
174,3,late,FX,56.1,2,1,10,23,Kiari,4,...,1,1,0,0,0,0,0,0,0,0
60,4,early,VKS,19.5,0,0,0,2,Guigo,0,...,0,0,0,0,0,1,0,0,0,0


In [165]:
# For each game, add a new line with the value of game_time == "pre-game" and each numeric column with value 0
early_games = data[data['game_time'] == 'early'].copy()
for column in early_games.columns:
    if column not in ['game','game_time', 'team_winner', 'blue_team', 'red_team', 
                      'blue_team_top', 'blue_team_jungle', 'blue_team_mid', 'blue_team_bottom', 'blue_team_support',
                      'red_team_top', 'red_team_jungle', 'red_team_mid', 'red_team_bottom', 'red_team_support']:
        early_games[column] = 0
    if column == 'game_time':
        early_games[column] = 'pre-game'

# Concatenate the original dataframe with the early_games dataframe
data = pd.concat([data, early_games], ignore_index=True)

In [166]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 87 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game                       500 non-null    int64  
 1   game_time                  500 non-null    object 
 2   blue_team                  500 non-null    object 
 3   blue_team_gold             500 non-null    float64
 4   blue_team_inhibitors       500 non-null    int64  
 5   blue_team_barons           500 non-null    int64  
 6   blue_team_towers           500 non-null    int64  
 7   blue_team_kills            500 non-null    int64  
 8   blue_team_top              500 non-null    object 
 9   blue_team_top_kills        500 non-null    int64  
 10  blue_team_top_deaths       500 non-null    int64  
 11  blue_team_top_assists      500 non-null    int64  
 12  blue_team_top_cs           500 non-null    int64  
 13  blue_team_top_gold         500 non-null    float64

In [167]:
# Seleciona aleatoriamete 30 partidas
game_range = np.random.randint(1, 130, 30)

# Seleciona as partidas com a coluna game no range selecionado
data_sample = data[data['game'].isin(game_range)]

print('Partidas selecionadas: ', game_range)

Partidas selecionadas:  [ 85  62  17  30 110  81  68 108  77  17 114  69   3 105  77  78   7  46
  70 118  45  85  57  92  76   1  69  51 105  39]


In [168]:
data_train = data[data['game'].isin(game_range) == False]
data_test = data_sample[data_sample['game_time'] == 'pre-game']

In [169]:
# drop the 'game' column
data_train.drop('game', axis=1, inplace=True)
data_test.drop('game', axis=1, inplace=True)

C:\Users\Marlon\AppData\Local\Temp\ipykernel_15704\1229804565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop('game', axis=1, inplace=True)
C:\Users\Marlon\AppData\Local\Temp\ipykernel_15704\1229804565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop('game', axis=1, inplace=True)


In [170]:
numeric_features = data_test.select_dtypes(include=[np.number])
numeric_features.columns

Index(['blue_team_gold', 'blue_team_inhibitors', 'blue_team_barons',
       'blue_team_towers', 'blue_team_kills', 'blue_team_top_kills',
       'blue_team_top_deaths', 'blue_team_top_assists', 'blue_team_top_cs',
       'blue_team_top_gold', 'blue_team_jungle_kills',
       'blue_team_jungle_deaths', 'blue_team_jungle_assists',
       'blue_team_jungle_cs', 'blue_team_jungle_gold', 'blue_team_mid_kills',
       'blue_team_mid_deaths', 'blue_team_mid_assists', 'blue_team_mid_cs',
       'blue_team_mid_gold', 'blue_team_bottom_kills',
       'blue_team_bottom_deaths', 'blue_team_bottom_assists',
       'blue_team_bottom_cs', 'blue_team_bottom_gold',
       'blue_team_support_kills', 'blue_team_support_deaths',
       'blue_team_support_assists', 'blue_team_support_cs',
       'blue_team_support_gold', 'red_team_gold', 'red_team_inhibitors',
       'red_team_barons', 'red_team_towers', 'red_team_kills',
       'red_team_top_kills', 'red_team_top_deaths', 'red_team_top_assists',
       'r

In [171]:
#change dtype object to category
for col in data_test.columns:
    if data_test[col].dtype == 'object':
        data_test[col] = data_test[col].astype('category')

for col in data_train.columns:
    if data_train[col].dtype == 'object':
        data_train[col] = data_train[col].astype('category')

C:\Users\Marlon\AppData\Local\Temp\ipykernel_15704\3757224261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test[col] = data_test[col].astype('category')
C:\Users\Marlon\AppData\Local\Temp\ipykernel_15704\3757224261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test[col] = data_test[col].astype('category')
C:\Users\Marlon\AppData\Local\Temp\ipykernel_15704\3757224261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [172]:
category_features = data_train.select_dtypes(include=['category'])
category_features.drop(['team_winner'], axis=1, inplace=True)
category_features.columns

Index(['game_time', 'blue_team', 'blue_team_top', 'blue_team_jungle',
       'blue_team_mid', 'blue_team_bottom', 'blue_team_support', 'red_team',
       'red_team_top', 'red_team_jungle', 'red_team_mid', 'red_team_bottom',
       'red_team_support'],
      dtype='object')

In [173]:
# from sklearn.model_selection import train_test_split

# RANDOM_SEED = 42
# np.random.seed(RANDOM_SEED)

# X = data.drop('team_winner', axis=1)
# y = data['team_winner']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

X_train = data_train.drop('team_winner', axis=1)
y_train = data_train['team_winner']

X_test = data_test.drop('team_winner', axis=1)
y_test = data_test['team_winner']

print(X_train.shape)
print(X_test.shape)

(400, 85)
(25, 85)


In [174]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [176]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

# Assuming X_train and X_test are already defined as your training and testing data
numeric_features = X_train.select_dtypes(include=[np.number]).columns
categorical_features = X_train.select_dtypes(include=['category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(categories='auto', drop='first', sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [177]:
#Modelagem usando XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_model = XGBClassifier(n_estimators=1000, max_depth=3, learning_rate=0.1, random_state=RANDOM_SEED)
xgb_model.fit(X_train_preprocessed, y_train)

y_pred_xgb = xgb_model.predict(X_test_preprocessed)

print('Accuracy Score: {}'.format(accuracy_score(y_test, y_pred_xgb)))

Accuracy Score: 0.4


In [178]:
from sklearn.model_selection import GridSearchCV

#Modelagem usando XGBoost com tuning de hiperparâmetros
param_grid_xgb = {
    'n_estimators': [100, 200, 1000],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_model = XGBClassifier(random_state=RANDOM_SEED)

gs_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=5, n_jobs=-1, verbose=2)
gs_xgb.fit(X_train_preprocessed, y_train)

gs_xgb.best_params_

Fitting 5 folds for each of 27 candidates, totalling 135 fits


c:\Users\Marlon\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 1000}

In [179]:
gs_xgb.best_score_

0.7525000000000001

In [180]:
from sklearn.utils import resample

#Bootsrap usando o modelo XGboost para a multiclasses
model_xgb = XGBClassifier(n_estimators=1000, max_depth=3, learning_rate=0.2, random_state=RANDOM_SEED)
model_xgb.fit(X_train_preprocessed, y_train)

# bootstrap predictions
accuracy_xgb = []
n_iterations = 1000

for i in range(n_iterations):
    X_bs, y_bs = resample(X_train_preprocessed, y_train, replace=True)
    # make predictions
    y_hat = model_xgb.predict(X_bs)
    # evaluate model
    score = accuracy_score(y_bs, y_hat)
    accuracy_xgb.append(score)

In [181]:
#Show metrics of bootstrap
print('Mean Accuracy: {:.3f}'.format(np.mean(accuracy_xgb)))
print('Standard Deviation Accuracy: {:.3f}'.format(np.std(accuracy_xgb)))

Mean Accuracy: 0.980
Standard Deviation Accuracy: 0.007


In [182]:
# predict the result of a new game 

game_sample = data[data['game_time'] == 'pre-game'].copy()
#copy the first line of the dataframe
new_game = game_sample.iloc[3].copy()
#show the blue team and red team
print('Blue Team: {}'.format(new_game['blue_team']))
print('Red Team: {}'.format(new_game['red_team']))
print('Winner: {}'.format(new_game['team_winner']))
new_game_test = new_game.drop('team_winner')


Blue Team: VKS
Red Team: LOS
Winner: VKS


In [183]:
# transform the new game to the same format of the training data
new_game_test = pd.DataFrame(new_game_test).T
new_game_test = preprocessor.transform(new_game_test)

# predict the result of the new game
y_pred = model_xgb.predict(new_game_test)

# transform the result to the original label
y_pred = le.inverse_transform(y_pred)

print('The winner of the game is: {}'.format(y_pred[0]))

The winner of the game is: VKS
